**классификатор текстов LSTM на Keras+TensorFlow**

Евгений Борисов <borisov.e@solarl.ru>

https://habr.com/ru/company/dca/blog/274027/    
http://help.sentiment140.com/for-students/   
http://study.mokoron.com  

## Библиотеки

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 200  
import re
import gzip

In [2]:
def pp(d): return "{:,.0f}".format(d).replace(",", " ")
def ppr(d): print('записей:', pp(len(d)) )  

## Данные

In [3]:
ff = ['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount','unk']

In [4]:
neg = pd.read_csv('../data/text/twit/negative.csv.gz',sep=';',header=None)
ppr(neg)
neg.columns = ff

записей: 111 923


In [5]:
pos = pd.read_csv('../data/text/twit/positive.csv.gz',sep=';')
ppr(pos)
pos.columns = ff

записей: 114 910


In [6]:
data = pd.concat([pos,neg],sort=False)[['id','ttext', 'ttype']]
ppr(data)

записей: 226 833


In [7]:
data.sample(10)

,id,ttext,ttype
2001,408927811198394368,"@OlyaDemenk вот ещё что вспомнила,ты там книгу не забудь,о которой я тебе говорила :)",1
78841,410725013625401345,@schastlivaya87 на тебя #ГрустьПечальТоска напала? ),1
98512,423009421330235392,@_alex_lol блин а я обновила на айпаде у меня не так как у тебя:(((,-1
24492,409545001182691328,"Лучше пусть будут у тебя 2 друга, но хороших, чем 10 предателей.)) http://t.co/ZusgC7XJfW",1
30627,409644539444727808,RT @TukvaSociopat: Специально для российских СМИ! На Майдане больше 1 000 000 Свободных Граждан Украины! ))) #євромайдан http://t.co/29TeaY…,1
49785,415113597069852672,Пол часа просидела в автошколе и уже утомилась:(,-1
77728,410712896365395968,"@wladimir_krun только собиралась позвонить по приезду домой, а ты уже увидел. А я все равно позвоню)",1
108864,411168372664307712,Очень страшное зрелище-ванна после присутствия там голубой косметической глины...)) http://t.co/puZ587qzPK,1
28041,409609789153746944,"RT @Dm_Agletdinov: 180 человек, это было круто, ну да вроде:D http://t.co/2KACpfSXza",1
50108,410034830185754624,угадайте кто сегодня ушел с второга урока и проебал кр по инглишу и химии),1


## очистка данных

In [8]:
data['ttext_clean'] = data['ttext'].apply(lambda t:[ w.strip() for w in t.split() if w.strip() ] )

In [9]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'^http.*',' url ', w.strip() ) for w in t  ]
  )

In [10]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'[:;]-*[)D]',' happysmile ', w.strip() )for w in t ]
  )

In [11]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'\)\)\)*',' happysmile ', w.strip() ) for w in t ]
  )

In [12]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'[:;]\*',' kisssmile ', w.strip() ) for w in t ]
  )

In [13]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r':\(',' sadsmile ', w.strip() ) for w in t ]
  )

In [14]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t:[ re.sub(r'\(\(\(*',' sadsmile ', w.strip() ) for w in t ]
  )

In [15]:
data['ttext_clean'] = [ ' '.join(s) for s in data['ttext_clean'] ]

In [16]:
data['ttext_clean'] = data['ttext_clean'].str.lower()
data['ttext_clean'] = data['ttext_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
data['ttext_clean'] = data['ttext_clean'].apply(lambda s: re.sub( r'_', ' ', s))
data['ttext_clean'] = data['ttext_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' digit ', s)) 


In [17]:
data['ttext_clean'] = data['ttext_clean'].apply(lambda t:[ w.strip() for w in t.split() if w.strip() ] )

In [18]:
# замена буквенно-цифровых кодов
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t: [w for w in t if not re.match( r'\b.*\d+.*\b', w) ]
)

In [19]:
# data[['ttext_clean']]
# data[['ttext']]

---

In [20]:
# from nltk import download as nltk_download
# nltk_download('stopwords')

# from Stemmer import Stemmer

from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords

stopwords = set(nltk_stopwords.words('russian') )

In [21]:
# with gzip.open('../data/text/stop-nltk.txt.gz','rt',encoding='utf-8') as f: 
#     stopwords = set([ w.strip() for w in  f.read().split() if w.strip() ] )

ppr(stopwords)

записей: 151


In [22]:
# удаление лишних слов
data['ttext_clean'] = data['ttext_clean'].apply(lambda t:[w for w in t if w not in stopwords])

In [23]:
%xdel stopwords

In [24]:
# %%time 

# from Stemmer import Stemmer
# # pacman -S python-pystemmer
# # pip install pystemmer

# # стемминг, выделение основы слова
# data['ttext_clean'] = data['ttext_clean'].apply( lambda t:Stemmer('russian').stemWords(t) )

In [25]:
# удаление коротких слов
data['ttext_clean'] = data['ttext_clean'].apply(lambda t:[w for w in t if len(w)>2])

---

In [26]:
# data[ data['ttext_clean'].str.len()<1 ][['ttext_clean']]

In [27]:
ppr(data)
data = data[ data['ttext_clean'].str.len()>0 ].reset_index(drop=True) 
ppr(data)

записей: 226 833
записей: 226 826


In [28]:
data.sample(3)

,id,ttext,ttype,ttext_clean
138017,411897104139702272,"@tsareva_k мам, у папки депресняк какой та((9(9;",-1,"[tsareva, мам, папки, депресняк, sadsmile, digit, digit]"
213979,423043956394315776,"Судьба-эт мост,который строишь к тому,кого ты любишь...(((",-1,"[судьба, мост, который, строишь, тому, кого, любишь, sadsmile]"
183011,417350736108720128,"завтра рано вставать, а мне блять спать не хочется((((",-1,"[завтра, рано, вставать, блять, спать, хочется, sadsmile]"


## строим датасет

In [29]:
vocab = ['<PAD>','<START>','<UNK>'] + sorted(set([ w for t in data['ttext_clean'] for w in t if w ]))
ppr(vocab)

записей: 239 405


In [30]:
# %%time

# from gensim.models.word2vec import Word2Vec

# w2v = Word2Vec( common_texts, min_count=1, size=256, window=4, workers=4)

# # with open('result/Word2Vec.pkl', 'wb') as f: pickle.dump(w2v, f)

In [31]:
vocab = { w:n for n,w in enumerate(vocab) }

---

In [32]:
data['ttext_clean'] = data['ttext_clean'].apply( lambda d: d+['<START>'] )

In [33]:
n_max = data['ttext_clean'].str.len().max()
n_max

31

In [34]:
pad = ['<PAD>']*n_max

In [35]:
data[['ttext_clean']]

,ttext_clean
0,"[таки, немного, похож, мальчик, равно, happysmile, <START>]"
1,"[katiacheh, идиотка, испугалась, <START>]"
2,"[углу, сидит, погибает, голода, ещё, digit, порции, взяли, хотя, жрать, хотим, happysmile, url, <START>]"
3,"[irina, dyshkant, значит, страшилка, happysmile, блин, посмотрев, части, создастся, ощущение, авторы, курили, happysmile, <START>]"
4,"[любишь, знаю, бля, happysmile, url, <START>]"
...,...
226821,"[каждый, хочет, исправлять, sadsmile, url, <START>]"
226822,"[скучаю, taaannyaaa, вправляет, мозги, равно, скучаю, <START>]"
226823,"[школу, говно, это, идти, <START>]"
226824,"[them, lisaberoud, тауриэль, грусти, sadsmile, обнял, <START>]"


In [36]:
data['ttext_clean'] = data['ttext_clean'].apply(
    lambda t: pad[len(t):] + list(reversed(t)) 
  )

In [37]:
data[['ttext_clean']]

,ttext_clean
0,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, happysmile, равно,..."
1,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START..."
2,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, url, happysmile, хотим, жрать, хотя, взяли, порции, digit, ещё, гол..."
3,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, happysmile, курили, авторы, ощущение, создастся, части, посмотрев, ..."
4,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, url, happys..."
...,...
226821,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, url, sadsmi..."
226822,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, скучаю, равно, моз..."
226823,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, идти..."
226824,"[<PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <START>, обнял, sadsmile, г..."


In [38]:
data['ttext_code'] = data['ttext_clean'].apply(lambda t: [ vocab[w] for w in t ] )

In [39]:
data['ttext_code'].values

array([list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 21934, 187622, 136470, 178005, 149250, 211202]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 119862, 117104, 27796]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 61233, 21934, 228320, 108687, 228340, 85322, 176263, 13807, 107122, 95445, 170005, 198948, 218486]),
       ...,
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 117142, 236937, 95093, 234034]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 154236, 51507, 97204, 211977, 33808, 58781]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 51507, 133603, 236866, 187340, 89864, 181790, 188632, 187566, 83969, 211277])],
      dtype=object)

In [40]:
len(data)//32

7088

In [41]:
ppr(data)
data = data.sample(32*7088).reset_index(drop=True)
ppr(data)


записей: 226 826
записей: 226 816


---

In [42]:
X = np.stack( data['ttext_code'].values).astype(np.float32 ) # , axis=-1)
X.shape

(226816, 31)

In [43]:
from sklearn.preprocessing import OneHotEncoder

y = data['ttype'].values
y = OneHotEncoder(categories='auto').fit_transform(y.reshape(-1,1) ).todense().astype(np.float32)
y.shape


(226816, 2)

In [44]:
# np.save('X.npy',X)
# np.save('y.npy',y)

In [45]:
# import numpy as np

# X = np.load('X.npy')
# y = np.load('y.npy')

In [46]:
vocab_size = int(X.max())
X.shape , y.shape, vocab_size

((226816, 31), (226816, 2), 239404)

## строим нейросеть 

In [47]:
# import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [48]:
# n=226826
# for i in range(1,n//2):
#     if n%i==0: print(i)
# # 23
# # 46
# # 4931
# # 9862

In [49]:
time_steps=X.shape[1]
batch_size=32
num_classes=y.shape[1]

vocab_size = len(vocab)

In [50]:
embedding_size=64

model = Sequential()

model.add(Embedding(
       input_dim=vocab_size, # e.g, 10 if you have 10 words in your vocabulary
       output_dim=embedding_size, # size of the embedded vectors
       input_length=time_steps,
       batch_input_shape=(batch_size,time_steps)
    ))

model.add(LSTM(
       32, 
       return_sequences=False, 
       stateful=False)
    )

model.add(Dense(num_classes, activation='softmax'))

In [51]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [53]:
%%time

model.fit(X,y, batch_size=batch_size, epochs=1, )

Train on 226816 samples
226816/226816 [==============================] - 697s 3ms/sample - loss: 0.2145 - accuracy: 0.8882
CPU times: user 9min 19s, sys: 11min 7s, total: 20min 26s
Wall time: 11min 37s


In [55]:
# score = model.evaluate(X,y)